# Boosted Tree Models

## Another ensemble

Remember from last lesson that an **ensemble** is a collection of models that are combined under a single decision mechanism to model a specific outcome of interest. Our first example, **Random Forests**, is designed as an analogue to computer democracy. It improves the accuracy of the **Decision Tree** model by using a **bagging** (bootstrap aggregation) technique in addition to randomization. These randomized trees then create a more complete picture of the decision space, enabling a Random Forest to improve on the accuracy of a Decision Tree while also reducing the likelihood of overfitting.

This week, our model will not rely on the bagging technique. Instead, we will use **boosting** to improve the accuracy of our model while sticking with a tree-based model. As we will see, this will often improve performance to a greater extent than bagging or Random Forest models, but does not insulate against overfitting.

## Understanding Boosting

How is boosting different than bagging? Where bagging focuses on simultaneous and distinct models, boosting is focused on stacking models, so that one model provides input to the next model, and the process continues through all of the ensemble models. Let's take a closer look at how this can actually benefit us.

### The math

We can start off by describing what we aim to accomplish. Our goal with any machine learning algorithm is to estimate the underlying functional form ($f(x)$) of our variable of interest. The true functional form can't be known unless we fully describe the data generating process (unlikely), so instead, we build a model of that function based on observable characteristics. Let's start with a one stage model building our estimator of $f(x)$, which we will write as $\hat{f}_M(x)$, where $M$ is the stage of our model.

$$ \hat{f}_0(x) = 0 $$

Well, that's not very interesting...

Why is our 0-stage model just 0? To create a boosting algorithm, each individual stage in our model is constructed with the aim of explaining the error in the previous stage's model. In order for our 1-stage model to be able to fit this mold, we build our 0-stage, or baseline, model as a uniform predictor. It will simply make the same prediction independent of the value of $x$. Our first stage model is then designed to predict the error in the 0-stage model.

$$ \hat{f}_1(x) = \hat{f}_0(x) - 0 $$

We can then write our one stage model as

$$ \hat{f}(x) = \hat{f}_0(x) - \hat{f}_1(x) $$

If $M$ > 1, we simply continue to add stages, so that at stage $m$:

$$ \hat{f}_m(x) = \hat{f}_{m-1}(x) - \hat{f}^*_m(x) $$

where $\hat{f}^*_m(x)$ is the current prediction of last stage's error term.

### Visually

Let's look at a simple visual example found [here](https://developers.google.com/machine-learning/decision-forests/intro-to-gbdt). The visual below contains three panels. The leftmost panel is our current "complete" model, comprising all of our previous models for relevant values of $x$. In stage 0, though, this is just a straight line, since our model is naïve. The second panel is a plot of the current error term across values of $x$. The third panel represents the current stage's tree-based model of the error term.

Together, these plots represent the process of fitting the next stage in a boosted tree model. We take our current model, map out the error of that model, and then fit a new model to the current error "function".

![](https://developers.google.com/static/machine-learning/decision-forests/images/ThreePlotsAfterFirstIteration.png)

We can see even more clearly what is happening when we look at this figure in combination with the same figure for the *next* stage. Now, our right-hand panel/model has been flipped upside down and combined with our previous "complete" model (this is why we wrote the "additive" nature of our boosted tree model as subtraction rather than as addition) to form the left panel of the next stage. Again, the middle panel is the current stage's error function, and the right panel is a tree-based model of the current error.

![](https://developers.google.com/static/machine-learning/decision-forests/images/ThreePlotsAfterSecondIteration.png)

Between the right panel of the 0 stage model, and the left panel of the stage 1 model, we can see that the predictions of our outcome of interest are simply the flipped predictions of the errors from the previous stage. Why? Because the error is how far off we were last time. If we don't want to be that far off, we should simply subtract the error from our past predictions from our those predictions to get a better set of predictions. The error term in stage 1 is smaller than in stage 0 (we've corrected some of the stage 0 model's biggest errors), and so the additive component in our new model (the right panel) is now correcting the largest remaining errors.

When we advance to the next stage, we will see that the right hand panel will be combined (upside down) with the left panel to create the new model to start the next stage. This will continue throughout each stage of our boosted model, which slowly gains in accuracy as we compound each of the simple error-prediction models to create our true model of the data generation process.

![](https://developers.google.com/static/machine-learning/decision-forests/images/ThreePlotsAfterTenIterations.png)

In this particular example, the stage 10 model actually closely resembles the overall data-generation process that we could observe as the error term in stage 0. While it is blocky and not smooth, it is shaped in much the same way as the wave function it was meant to predict. This same process will hold as we use boosted tree models for continuous dependent variables (as we did above), as well as when we attempt to predict outcomes in a classification context, as we will do in our example below.

## Implementing boosting with `xgboost`

To implement our boosted tree model, we are going to use the `xgboost` library. It's a nice library for a few reasons. First, it's available in multiple languages (R, Python, Julia, etc.) so you can learn one model and then be able to use it wherever you find yourself. Second, the `xgboost` model has been carefully optimized to run as efficiently as possible, including being able to take advantage of GPUs or parallel computation when training larger models.

To get started, let's load our libraries and the MNIST training dataset.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = pd.read_csv("https://github.com/dustywhite7/Econ8310/raw/master/DataSets/mnistTrain.csv")

# Upper case before split, lower case after
Y = data['Label']
# make sure you drop a column with the axis=1 argument
X = data.drop('Label', axis=1) 

x, xt, y, yt = train_test_split(X, Y, test_size=0.1,
     random_state=42)

Our data has ten classes, so we are doing classification, but we need to be able to predict more than just "success" or "failure". Any integer between 0 and 9 is a valid prediction.

Let's prepare our model accordingly.

In [2]:
# Run this cell if you need to install xgboost
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.8/124.9 MB 2.6 MB/s eta 0:00:48
   ---------------------------------------- 1.3/124.9 MB 2.6 MB/s eta 0:00:48
    --------------------------------------- 1.8/124.9 MB 2.3 MB/s eta 0:00:53
    --------------------------------------- 2.1/124.9 MB 2.3 MB/s eta 0:00:53
    --------------------------------------- 2.6/124.9 MB 2.3 MB/s eta 0:00:54
   - -------------------------------------- 3.4/124.9 MB 2.5 MB/s eta 0:00:50
   - -------------------------------------- 3.9/124.9 MB 2.5 MB/s eta 0:00:48
   - -------------------------------------- 4.7/124.9 MB 2.6 MB/s eta 0:00:46
   - -------------------------------------- 5.2/124.9 MB 2.6 MB/s eta 0:00:46
   - -------------------------------------- 5.8/124.9 MB 2.7 MB/s eta 0:00:45
   - -------------------------------------- 6.0/124.9 MB 2.7 MB/s eta 0:00:45


In [3]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=50, 
    max_depth=3,
    learning_rate=0.5, 
    objective='multi:softmax'
)

Some of these arguments will look familiar, because `xgboost` tries to adhere as much as possible to the `sklearn` API patterns. Just like our Random Forests, we specify `n_estimators` for our ensemble model. In this case, this value does not represent the number of parallel trees that will be voting. It instead represents the number of stages or iterations to which our model will be subjected.

`max_depth` specifies the maximum depth of the tree at each stage that will be used to model the current error function.

`learning_rate` is a weight that is applied to each stage in an attempt to prevent overfitting. A learning rate of less than one reduces the impact of a given stage's weight on the overall model, so that the model converges more slowly toward the underlying function, and the chance of overshooting is reduced.

Fitting the model is just as simple as it was for our `sklearn` models.

In [5]:
# Fit to our training split
xgb.fit(x, y)

# Make predictions based on the testing x values
pred = xgb.predict(xt)

# Print out the accuracy score
print(f"Accuracy score: {accuracy_score(yt, pred)*100:.2f}%")

Accuracy score: 93.40%


A very nice accuracy level for such a simple model! From here, we can explore whether other combinations of hyperparamters increase our model performance or not. In order to do that, we should spend some time talking about cross validation.

# Cross-Validation

In order to maximize the information that we have about our model's performance, we will often test the model on many draws of our existing data. We do this because the random samples that we draw from our data can actually have a significant impact on how we interpret our model's performance. Some draws will have outliers in the training set, so that we build a model that accomodates them, while other draws may leave outliers only in our testing draw. A process of taking multiple samples to explore how our model performs on different subsets of data is called **cross-validation**. We resample our training and testing data $k$ times, and compare the performance of the models, so that we can gain a better picture of how sensitive our model may be to including or excluding different draws of data.

If the models for each of our draws perform more or less equally well, then we can treat our model as well-specified. If not, then we know performance was sample-dependent.

## Implementation

We often call cross-validation "$k$-fold cross-validation", where $k$ is the number of groups into which we will sample our data. The image below (from the sklearn documentation) depicts a 5-fold cross-validation. Steps to accomplish this are as follows:

1) Sample data into training and testing data. The testing data will not be involved in the cross-validation, since it will be used as a final validity check on our model **after** we do our cross-validation.
2) In our training data, we split our data into $k$ (in this case, 5) folds. Each fold will be of equal size, and each observation is only drawn once. We will use each observation in our training data during this exercise.
3) Once we have all of our folds, we run our model $k$ times. In the first iteration, we take $k-1$ folds of our data as training data, with the remaining fold serving as our testing data. We train the model, then test and store our model and performance metrics.
4) Repeat step (3), withholding each of the folds once in turn.
5) Compare the performance of the models.

![](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In the case of cross-validation, we are not looking for the BEST model. We are instead looking for consistency across models. A well specified model should perform similarly in each instance, and we want to be able to minimize the variation across our folds. By doing this, we ensure that we have an accurate perception of how our model will perform when it encounters new data.

If we are not happy with our models performance, we can try a new model and run cross-validation again. If we are satisfied, then we can move on to training our model on the entire training dataset. Having done so, we are ready to test our model against the real test data, where we should expect our model to once again perform at the same level as it did during cross-validation. If it does not, this can serve as another red flag indicating overfitting.

Below is some example code of reporting accuracy of models within a cross-validation loop.

In [6]:
from sklearn.model_selection import KFold

# If we have imported data and created x, y already:
kf = KFold(n_splits=10) # 10 "Folds"

models = [] # We will store our models here

for train, test in kf.split(x): # Iterate over folds
    model = XGBClassifier(
            n_estimators=50, 
            max_depth=3,
            learning_rate=0.5, 
            objective='multi:softmax'
                ).fit(x.values[train], y.values[train]) # Fit model
    accuracy = accuracy_score(y.values[test],    # Store accuracy
        model.predict(x.values[test]))
    print("Accuracy: ", accuracy)            # Print results
    models.append([model, accuracy])      # Store it all

print("Mean Model Accuracy: ", np.mean([model[1] for model in models]))
print("Model Accuracy Standar Deviation: ", np.std([model[1] for model in models]))

Accuracy:  0.9311111111111111
Accuracy:  0.9266666666666666
Accuracy:  0.9266666666666666
Accuracy:  0.9355555555555556
Accuracy:  0.8933333333333333
Accuracy:  0.9311111111111111
Accuracy:  0.9311111111111111
Accuracy:  0.9511111111111111
Accuracy:  0.9266666666666666
Accuracy:  0.9333333333333333
Mean Model Accuracy:  0.9286666666666668
Model Accuracy Standar Deviation:  0.013606461790970352


**Reading Reflection**:

Are you more comfortable with the tradeoffs of random forests or boosted trees? Explain your preference in 2-3 sentences.
